# SVM

In [5]:
!gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?code_challenge=5kHGtuc3vAXO-l-9s3alDQW6ghKp6lZ65LXApWJ8_-Y&prompt=select_account&code_challenge_method=S256&access_type=offline&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&response_type=code&client_id=32555940559.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth


If you need to use ADC, see:
  gcloud auth application-default --help

You are now logged in as [galli.giuly@gmail.com].
Your current project is [reddit-master].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [2]:
!gcloud config set project reddit-master

Updated property [core/project].


In [4]:
import pandas as pd
import numpy as np
import pickle as pkl
import nltk.data
import joblib
import nltk
import ast
import logging


from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
import datetime
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/pepe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
def loggg(msg):
    print("[INFO] {}: {}".format(datetime.datetime.now(), msg))

In [6]:
red_model_SVM_df = pd.read_pickle("red_comments_posts_tokenized.pkl")

In [7]:
red_model_SVM_df.head()

,subreddit,body,subreddit_id
0,funny,"[hell, job, blue, duck, photo, sourc]",5
1,funny,"[one, settl]",5
2,funny,"[grand, father, fish, cod, like, 50, year, ago]",5
3,funny,"[fake, yes, , effect, real]",5
4,funny,"[smoke, bomb, bug, poison, combin]",5


In [8]:
X = red_model_SVM_df['body']
y = red_model_SVM_df['subreddit_id']

# Definint a fucntion that slit the dataset into three subsets: train, val and test
def train_dev_test_split(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    return dict(X_train=X_train,
                X_test=X_test,
                y_train=y_train.astype('int'),
                y_test=y_test.astype('int'))

In [9]:
red_model_SVM_df_split = train_dev_test_split(X,y)

In [26]:
parameters = {
        "tfidf__max_df": [0.25, 0.5, 0.75],
        "tfidf__ngram_range": [(1, 2)],
        "clf__C": [0.01, 0.1, 1],
        "clf__class_weight": ["balanced"]   
}

In [27]:
%%time

pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(preprocessor=' '.join)),
            ('clf', LinearSVC()),
            ])

grid_search_tune = GridSearchCV(pipeline, parameters, cv=2, n_jobs=2, verbose=10)

CPU times: user 227 µs, sys: 91 µs, total: 318 µs
Wall time: 2.1 ms


In [28]:
loggg("pipeline created")

grid_search_tune.fit(red_model_SVM_df_split['X_train'], red_model_SVM_df_split['y_train'].to_numpy().astype('int'))

[INFO] 2019-11-30 11:29:30.716978: pipeline created
Fitting 2 folds for each of 9 candidates, totalling 18 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:  3.1min
[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:  6.3min
[Parallel(n_jobs=2)]: Done   9 tasks      | elapsed: 14.0min
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed: 22.7min
[Parallel(n_jobs=2)]: Done  18 out of  18 | elapsed: 31.5min finished


GridSearchCV(cv=2, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('tfidf',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                         

In [29]:
# measuring performance on test set
loggg('Best parameters set:')
print(grid_search_tune.best_estimator_.steps)
# measuring performance on test set
loggg('Applying best classifier on test data:')

[INFO] 2019-11-30 12:05:19.296374: Best parameters set:
[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.25, max_features=None,
                min_df=1, ngram_range=(1, 2), norm='l2',
                preprocessor=<built-in method join of str object at 0x7efc147d4970>,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)), ('clf', LinearSVC(C=0.1, class_weight='balanced', dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0))]
[INFO] 2019-11-30 12:05:19.297739: Applying best classifier on test data:


In [31]:
best_clf = grid_search_tune.best_estimator_
predictions = best_clf.predict(red_model_SVM_df_split['X_test'])
print(classification_report(red_model_SVM_df_split['y_test'], predictions))

              precision    recall  f1-score   support

           0       0.74      0.87      0.80     46567
           1       0.45      0.36      0.40     46567
           2       0.57      0.62      0.60     46567
           3       0.55      0.67      0.60     46097
           4       0.56      0.59      0.57     46567
           5       0.33      0.26      0.29     46567
           6       0.60      0.61      0.61     46567
           7       0.62      0.68      0.65     46568
           8       0.76      0.82      0.78     46567
           9       0.48      0.50      0.49     46257
          10       0.46      0.57      0.50     46125
          11       0.51      0.57      0.54     46164
          12       0.35      0.19      0.25     46093
          13       0.40      0.28      0.33     46098

    accuracy                           0.54    649371
   macro avg       0.53      0.54      0.53    649371
weighted avg       0.53      0.54      0.53    649371



In [33]:
with open('red_model_svm.pkl', 'wb') as file:
    pkl.dump(best_clf, file)

In [34]:
!gsutil cp red_model_svm.pkl gs://reddit_models/

Copying file://red_model_svm.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

\ [1 files][  1.7 GiB/  1.7 GiB]    3.8 MiB/s                                   
Operation completed over 1 objects/1.7 GiB.                                      
